## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [12]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Mataura,-46.1927,168.8643,42.01,83,100,5.99,NZ,overcast clouds
1,1,Ostrovnoy,68.0531,39.5131,24.62,84,100,6.78,RU,overcast clouds
2,2,San Patricio,28.0170,-97.5169,82.00,83,90,13.80,US,overcast clouds
3,3,Avarua,-21.2078,-159.7750,77.00,94,97,4.61,CK,overcast clouds
4,4,Bijie,27.3125,105.2942,56.62,93,100,2.98,CN,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
ideal_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

In [13]:
# 4a. Determine if there are any empty rows.
print(ideal_cities_df.value_counts().sum())
print(ideal_cities_df.shape)

193
(193, 10)


In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = ideal_cities_df.dropna()
clean_df.head()

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
2,2,San Patricio,28.0170,-97.5169,82.00,83,90,13.80,US,overcast clouds
3,3,Avarua,-21.2078,-159.7750,77.00,94,97,4.61,CK,overcast clouds
8,8,Hilo,19.7297,-155.0900,75.20,60,1,6.38,US,clear sky
11,11,Georgetown,5.4112,100.3354,80.60,89,40,2.30,MY,scattered clouds
14,14,Atuona,-9.8000,-139.0333,79.68,70,90,16.84,PF,overcast clouds


In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,San Patricio,US,82.00,overcast clouds,28.0170,-97.5169,
3,Avarua,CK,77.00,overcast clouds,-21.2078,-159.7750,
8,Hilo,US,75.20,clear sky,19.7297,-155.0900,
11,Georgetown,MY,80.60,scattered clouds,5.4112,100.3354,
14,Atuona,PF,79.68,overcast clouds,-9.8000,-139.0333,
16,Saint-Philippe,RE,75.20,light rain,-21.3585,55.7679,
18,Kapaa,US,75.99,clear sky,22.0752,-159.3190,
24,Ternate,ID,80.44,clear sky,0.8000,127.4000,
25,Ahuimanu,US,77.00,broken clouds,21.4447,-157.8378,
27,Merauke,ID,77.23,scattered clouds,-8.4667,140.3333,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat= row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat}, {lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print(hotels["results"][0]["name"])
    except (IndexError):
        print("Hotel not found... skipping")

Motel 6 Sinton, TX
Paradise Inn
Hilo Hawaiian Hotel
Cititel Penang
Villa Enata
Chambres d'hôte "La Trinité"
Sheraton Kauai Resort at Coconut Beach
Grand Dafam Bela Ternate
Paradise Bay Resort
Hotel not found... skipping
JABIR HOTEL
Lindsey Hôtel
Khách sạn Sài Gòn Bạc Liêu
Bungalow 63
Hotel not found... skipping
Hotel Kununurra
Shandrani Beachcomber Resort & Spa
Ted McQuinn Trust
Hotel not found... skipping
Namatanai Hotel
Casa Tia Villa
Pension Maro'i
Hotel not found... skipping
Scoop Guest House
Leumerus Bungalows
Amaris Hotel Citra Raya - Tangerang
Atlantis Hotel
Lorengau Harbourside Hotel
Orient Star Dahabyia Isna 05 Nights Each Sunday
STARiHOTELS Geedam Road
VS Villa
Pousada dos Navegantes
Uruna Bay Retreat
Shanti Maurice Resort & Spa
Isles Sunset Lodge
Veranda Paul et Virginie Hotel & Spa
Parra Hotel & Suites
Hotel JM Puerto del Rosario
Hampton Inn Yazoo City
Conquistador Hotel
Pousada Porto Praia
Hotel Santika Luwuk
Coral Sands Hotel
Mercure Padang
Haajy Guest House 1
Harrycane's

In [19]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"] !='')]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,San Patricio,US,82.00,overcast clouds,28.0170,-97.5169,"Motel 6 Sinton, TX"
3,Avarua,CK,77.00,overcast clouds,-21.2078,-159.7750,Paradise Inn
8,Hilo,US,75.20,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel
11,Georgetown,MY,80.60,scattered clouds,5.4112,100.3354,Cititel Penang
14,Atuona,PF,79.68,overcast clouds,-9.8000,-139.0333,Villa Enata
16,Saint-Philippe,RE,75.20,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
18,Kapaa,US,75.99,clear sky,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
24,Ternate,ID,80.44,clear sky,0.8000,127.4000,Grand Dafam Bela Ternate
25,Ahuimanu,US,77.00,broken clouds,21.4447,-157.8378,Paradise Bay Resort
29,Bandarbeyla,SO,83.98,overcast clouds,9.4942,50.8122,JABIR HOTEL


In [21]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))